# WIP: Quantum chemistry with Qiskit Nature: finding the interatomic distance of a hydrogen molecule

Introduction

[refs: Sharkey and qiskit-nature documentation]
[look at qiskit nature documentation for explanations]

## MoleculeInfo
We start by describing the molecule that we want to look at, in our case a hydrogen molecule:

In [6]:
from qiskit_nature.second_q.formats import MoleculeInfo

from quantum_algorithms.vqe.energy_calculation import GroundStateEnergyCalculation

distance = 0.735
molecule = MoleculeInfo(["H", "H"], [(0.0, 0.0, 0.0), (0.0, 0.0, distance)], charge=0, multiplicity=1)

* The symbols are for the two hydrogen atoms.
* We are going to vary the coordinates (in Angstrom) of the second hydrogen atom. We'll use 3 Angstrom as the initial point.
* The total charge of the hydrogen atom is 0.
* The multiplicity is 2 * S + 1, where S is the spin of the molecule due to unpaired electrons. The hydrogen molecule has no unpaired electrons, so the multiplicity is 1.

Next, we convert the molecule into electronic structure problem through the PySCFDriver:

In [2]:
from qiskit_nature.second_q.drivers import PySCFDriver

driver = PySCFDriver.from_molecule(molecule)
es_problem = driver.run()

PySCFDriver is a driver from the PySCF library for second quantization. Second quantization means converting the description of the molecule from using electronic wave functions to using Fock space. Electronic wave functions give the probability density to find an electron at a certain set of coordinates. Wave functions can be built up from eigenfunctions, functions with an allowed energy. Using these eigenfunctions, the electronic configuration can the also be described in Fock space, where we no longer look at the position we may find the electron, but at which one of the eigenstates the electron is in. [check and improve, eigenstates/functions]

In [6]:
fermionic_op = es_problem.hamiltonian.second_q_op()
n_particles = es_problem.num_particles
n_spatial_orbitals = es_problem.num_spatial_orbitals
nuclear_repulsion_energy = es_problem.nuclear_repulsion_energy

## QubitConverter
To do quantum chemistry calculations using quantum computing, operators on electrons need to be converted to operators on qubits. This is done with a QubitConverter object:

In [7]:
from qiskit_nature.second_q.mappers import ParityMapper

mapper = ParityMapper(num_particles=(1, 1))
qubit_op = mapper.map(fermionic_op)

* The specific mapper chosen uses the Jordan-Wigner transformation [why? how does it work?]. Other possible mappers are ParityMapper and BravyiKitaevMapper. [why not these? how do they work?]

## VQEUCCFactory
We are going to calculate the interatomic distance of hydrogen using the Variational Quantum Eigensolver (VQE) algorithm. VQE works in a loop, where each iteration has the following steps:
 * A quantum circuit (a sequence of qubit operations) is described by a set of parameters
 * The qubit values are measured after executing the quantum circuit.
 * Based on the qubit values, new parameters for the quantum circuit are chosen.

In our case the qubit values represent the energy of the hydrogen molecule. The goal of the loop is to find the parameters of the quantum circuit that give the lowest energy (ground state energy). The parameters of the quantum circuit model the state of the electrons of the hydrogen molecule. [how does it determine this?]


In [8]:
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

initial_state = HartreeFock(n_spatial_orbitals, n_particles, mapper)
ansatz = UCCSD(n_spatial_orbitals, n_particles, mapper, initial_state=initial_state)

The VQEUCCFactory makes a VQE instance.
* an Estimator instance is used to estimate the eigenvalues [how exactly? why?]
* the UCCSD constructs a multi-electron wave function as the ansatz, with the sum of single (S) and double (D) excitations [how exactly? what is UCC?]
* optimization in the loop is done by the Sequential Least SQuares Programming optimizer






VQEUCCFactory and Estimator, UCCSD, SLSQP
PySCFDriver
ActiveSpaceTransformer
GroundStateEigenSolver

In [17]:
from qiskit.primitives import Estimator

estimator = Estimator()  # todo: move to V2 Estimator once https://github.com/qiskit-community/qiskit-algorithms/issues/136 is fixed

/tmp/ipykernel_9/2561769255.py:5: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator() #BackendEstimatorV2(backend=backend)


In [18]:

from qiskit_algorithms.optimizers import COBYLA

optimizer = COBYLA(maxiter=1)

In [19]:
from qiskit_algorithms import VQE

algo = VQE(estimator, ansatz, optimizer)
result = algo.compute_minimum_eigenvalue(qubit_op)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from quantum_algorithms.vqe.energy_calculation import GroundStateEnergyCalculation

distances = np.arange(0.25, 2.0, 0.1)
calc = GroundStateEnergyCalculation(molecule, n_max_iterations=1000)
energies = []
for i, d in enumerate(distances):
    calc.update_last_atoms_coords((0, 0, d))
    energy = calc.run()
    energies.append(energy)
    print(f"#{i:0>2} at {d:.2f} Angstrom -> energy: {energy:.3f}")
    
fig, ax = plt.subplots()
ax.plot(distances, energies)
ax.set_xlabel("Atomic distance [Angstrom]")
ax.set_ylabel("Ground state energy [Hartree]")


#00 at 0.25 Angstrom -> energy: -0.312
#01 at 0.35 Angstrom -> energy: -0.789
#02 at 0.45 Angstrom -> energy: -0.998
#03 at 0.55 Angstrom -> energy: -1.093
#04 at 0.65 Angstrom -> energy: -1.130
#05 at 0.75 Angstrom -> energy: -1.137
#06 at 0.85 Angstrom -> energy: -1.128


In [3]:
from quantum_algorithms.vqe.distance_finder import DistanceFinder

# todo: print formatted

distance_finder = DistanceFinder(iteration_callback=print, atoms=("H", "H"))
distance = distance_finder.run()
print(f"=== The distance between the two H atoms of the hydrogen molecule is calculated to be {distance:.3f} Angstrom ===")

IterationInfo(coords=(0.0, 0.0, 3.0), ground_state_energy=np.float64(-0.9335960990484058))
IterationInfo(coords=(1.0, 0.0, 3.0), ground_state_energy=np.float64(-0.933376874926096))
IterationInfo(coords=(0.0, 1.0, 3.0), ground_state_energy=np.float64(-0.9333846418274852))
IterationInfo(coords=(0.0, 0.0, 4.0), ground_state_energy=np.float64(-0.933166574828394))
IterationInfo(coords=(-0.4163334092833842, -0.401583114010936, 2.184280995160862), ground_state_energy=np.float64(-0.9395643024412575))
IterationInfo(coords=(-0.44717949085306335, -0.4313363483552773, 1.1851997850994138), ground_state_energy=np.float64(-1.0274034109641161))
IterationInfo(coords=(0.4278348680986865, -0.43243575622804775, 0.7011040683960046), ground_state_energy=np.float64(-1.1154990170798156))
IterationInfo(coords=(0.7321707152318171, 0.32736404905142935, 0.1265747409922343), ground_state_energy=np.float64(-1.132986929562505))
IterationInfo(coords=(1.1649589062572627, -0.20499908057780547, -0.6009442483958701), gro